# Data preparation

In [11]:
import numpy as np
import pandas as pd
import plotly.express as px

# Athletes

In [12]:
dataset_athlete = pd.read_json('../olympic_athletes.json')
dataset_athlete.head()
dataset_athlete.shape
dataset_athlete.isnull().sum()

athlete_url                 0
athlete_full_name           0
games_participations        0
first_game                 22
athlete_year_birth       2456
athlete_medals          60552
bio                     53062
dtype: int64

# Hotes

In [13]:
dataset_hosts = pd.read_xml('../olympic_hosts.xml')
dataset_hosts.head()
dataset_hosts.shape
dataset_hosts.columns
dataset_hosts.drop(['index'], axis=1, inplace=True)
dataset_hosts.columns
dataset_hosts.isnull().sum()

game_slug          0
game_end_date      0
game_start_date    0
game_location      0
game_name          0
game_season        0
game_year          0
dtype: int64

# Medailles

In [14]:
dataset_medals = pd.read_excel('../olympic_medals.xlsx')
dataset_medals.shape
dataset_medals.isnull().sum()
dataset_medals.duplicated().sum()
dataset_medals.columns
dataset_medals = dataset_medals.rename({'slug_game': 'game_slug'}, axis=1)
dataset_medals.columns
dataset_medals.loc[(dataset_medals.duplicated())]
dataset_medals.loc[(dataset_medals.game_slug == 'paris-1900') & (dataset_medals.discipline_title == 'Polo')]
dataset_medals.loc[(dataset_medals.game_slug == 'london-1908') & (dataset_medals.discipline_title == 'Polo')]
dataset_medals.loc[(dataset_medals.game_slug == 'london-1908') & (dataset_medals.discipline_title == 'Hockey') & (
        dataset_medals.event_title == 'hockey men')]
dataset_medals.participant_title.unique()
dataset_medals.drop(['Unnamed: 0', 'participant_title', 'athlete_url'], axis=1, inplace=True)
dataset_medals.tail()

,discipline_title,game_slug,event_title,event_gender,medal_type,participant_type,athlete_full_name,country_name,country_code,country_3_letter_code
21692,Weightlifting,athens-1896,heavyweight - one hand lift men,Men,SILVER,Athlete,Viggo JENSEN,Denmark,DK,DEN
21693,Weightlifting,athens-1896,heavyweight - one hand lift men,Men,BRONZE,Athlete,Alexandros Nikolopoulos,Greece,GR,GRE
21694,Weightlifting,athens-1896,heavyweight - two hand lift men,Men,GOLD,Athlete,Viggo JENSEN,Denmark,DK,DEN
21695,Weightlifting,athens-1896,heavyweight - two hand lift men,Men,SILVER,Athlete,Launceston ELLIOT,Great Britain,GB,GBR
21696,Weightlifting,athens-1896,heavyweight - two hand lift men,Men,BRONZE,Athlete,Sotirios VERSIS,Greece,GR,GRE


# Results

In [15]:
dataset_results = pd.read_html('../olympic_results.html')[0]
dataset_results.head()
dataset_results.shape
dataset_results.isnull().sum()
dataset_results.duplicated().sum()
dataset_results.loc[(dataset_results.duplicated())]
dataset_results.value_unit.unique()
dataset_results.value_type.unique()
dataset_results.drop(['Unnamed: 0', 'athlete_url'], axis=1, inplace=True)
dataset_results = dataset_results.rename({'slug_game': 'game_slug'}, axis=1)
dataset_results.columns

Index(['discipline_title', 'event_title', 'game_slug', 'participant_type',
       'medal_type', 'athletes', 'rank_equal', 'rank_position', 'country_name',
       'country_code', 'country_3_letter_code', 'athlete_full_name',
       'value_unit', 'value_type'],
      dtype='object')

# Separate winter and summer games

In [17]:
jo_types = dataset_hosts[['game_slug', 'game_season', 'game_year']]
merged_hosts_results = dataset_results.merge(jo_types, on='game_slug')
merged_hosts_results.head()
merged_hosts_results.shape
merged_hosts_results.game_season.unique()

array(['Winter', 'Summer'], dtype=object)

In [18]:
summer_games_results = merged_hosts_results.loc[(merged_hosts_results.game_season == 'Summer')].copy()
winter_games_results = merged_hosts_results.loc[(merged_hosts_results.game_season == 'Winter')].copy()

In [19]:
summer_games_results.drop(['game_season'], axis=1, inplace=True)
summer_games_results.head()
summer_games_results.shape
summer_games_results.isnull().sum()
summer_games_results.duplicated().sum()
dopplers = summer_games_results.loc[(summer_games_results.duplicated())]
dopplers

,discipline_title,event_title,game_slug,participant_type,medal_type,athletes,rank_equal,rank_position,country_name,country_code,country_3_letter_code,athlete_full_name,value_unit,value_type,game_year
154361,Water Polo,Water Polo Women,antwerp-1920,GameTeam,NaN,NaN,NaN,1,Netherlands,NL,NED,NaN,2,SCORE,1920
155336,Sailing,8m mixed,stockholm-1912,GameTeam,NaN,NaN,NaN,5,Russian Federation,RU,RUS,NaN,0,POINTS,1912
157014,Shooting,trap 125 targets men,stockholm-1912,Athlete,GOLD,NaN,NaN,1,United States of America,US,USA,Jay Graham,96,POINTS,1912
157015,Shooting,trap 125 targets men,stockholm-1912,Athlete,SILVER,NaN,NaN,2,Germany,DE,GER,Alfred GOELDEL,94,POINTS,1912
157016,Shooting,trap 125 targets men,stockholm-1912,Athlete,BRONZE,NaN,NaN,3,Russian Federation,RU,RUS,Harry Blaus,91,POINTS,1912
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
161736,Sailing,Â½-1 Ton Race Two Open,paris-1900,GameTeam,NaN,"[('LETOT LETOT', 'https://olympics.com/en/athl...",NaN,DNF,France,FR,FRA,NaN,NaN,IRM,1900
161737,Sailing,Â½-1 Ton Race Two Open,paris-1900,GameTeam,NaN,"[('LETOT LETOT', 'https://olympics.com/en/athl...",NaN,DNF,France,FR,FRA,NaN,NaN,IRM,1900
161738,Sailing,Â½-1 Ton Race Two Open,paris-1900,GameTeam,NaN,"[('LETOT LETOT', 'https://olympics.com/en/athl...",NaN,DNF,France,FR,FRA,NaN,NaN,IRM,1900
161739,Sailing,Â½-1 Ton Race Two Open,paris-1900,GameTeam,NaN,"[('LETOT LETOT', 'https://olympics.com/en/athl...",NaN,DNF,France,FR,FRA,NaN,NaN,IRM,1900


# Calculate the total number of each type of medals for each games by country

In [20]:
summer_games_results.medal_type.unique()
summer_games_results['medal_type'] = summer_games_results['medal_type'].fillna('None')
summer_games_results['total_medals'] = summer_games_results['medal_type'].apply(lambda x: 0 if x == 'None' else 1)
summer_games_results['gold_medals'] = summer_games_results['medal_type'].apply(lambda x: 1 if x == 'GOLD' else 0)
summer_games_results['silver_medals'] = summer_games_results['medal_type'].apply(lambda x: 1 if x == 'SILVER' else 0)
summer_games_results['bronze_medals'] = summer_games_results['medal_type'].apply(lambda x: 1 if x == 'BRONZE' else 0)
summer_games_results
summer_games_results.country_name.unique()
historic_medalsbycountry = summer_games_results.groupby(['game_year', 'country_name']).agg(
    {'total_medals': 'sum', 'gold_medals': 'sum', 'silver_medals': 'sum', 'bronze_medals': 'sum'})
historic_medalsbycountry = historic_medalsbycountry.sort_values(by=['game_year', 'total_medals'],
                                                                ascending=[True, False]).reset_index()
historic_medalsbycountry

,game_year,country_name,total_medals,gold_medals,silver_medals,bronze_medals
0,1896,Greece,47,10,18,19
1,1896,United States of America,20,11,7,2
2,1896,Germany,13,6,5,2
3,1896,France,11,5,4,2
4,1896,Great Britain,7,2,3,2
...,...,...,...,...,...,...
2884,2020,"Virgin Islands, British",0,0,0,0
2885,2020,"Virgin Islands, US",0,0,0,0
2886,2020,Yemen,0,0,0,0
2887,2020,Zambia,0,0,0,0


# Calculate the total number of disciplines for each games by country

In [21]:
historic_sportsbycountry = summer_games_results.groupby(['game_year', 'country_name', 'discipline_title']).count()
historic_sportsbycountry
historic_sportsbycountry = historic_sportsbycountry[['event_title']].reset_index()
historic_sportsbycountry = historic_sportsbycountry.rename({'discipline_title': 'sports', 'event_title': 'epreuves'},
                                                           axis=1)
historic_sportsbycountry
historic_sportsbycountry = historic_sportsbycountry.groupby(['game_year', 'country_name']).agg({'sports': 'count'})
historic_sportsbycountry = historic_sportsbycountry.reset_index()
historic_sportsbycountry
historic_epreuvesbycountry = summer_games_results.groupby(['game_year', 'country_name', 'event_title']).count()
historic_epreuvesbycountry

historic_epreuvesbycountry = historic_epreuvesbycountry[['discipline_title']].reset_index()
historic_epreuvesbycountry = historic_epreuvesbycountry.rename(
    {'event_title': 'epreuves', 'discipline_title': 'participation'}, axis=1)
historic_epreuvesbycountry = historic_epreuvesbycountry.groupby(['game_year', 'country_name']).agg(
    {'epreuves': 'count'})
historic_epreuvesbycountry = historic_epreuvesbycountry.reset_index()
historic_epreuvesbycountry
historic_olympic_data = historic_medalsbycountry.merge(historic_sportsbycountry, on=['game_year', 'country_name'])
historic_olympic_data = historic_olympic_data.merge(historic_epreuvesbycountry, on=['game_year', 'country_name'])
historic_olympic_data = historic_olympic_data.sort_values(by=['game_year', 'total_medals'], ascending=[True, False])
historic_olympic_data

,game_year,country_name,total_medals,gold_medals,silver_medals,bronze_medals,sports,epreuves
0,1896,Greece,47,10,18,19,10,39
1,1896,United States of America,20,11,7,2,3,17
2,1896,Germany,13,6,5,2,7,27
3,1896,France,11,5,4,2,7,20
4,1896,Great Britain,7,2,3,2,9,20
...,...,...,...,...,...,...,...,...
2884,2020,"Virgin Islands, British",0,0,0,0,1,2
2885,2020,"Virgin Islands, US",0,0,0,0,1,1
2886,2020,Yemen,0,0,0,0,2,2
2887,2020,Zambia,0,0,0,0,3,5


# Save the data to ../csv/olympic_data_cleaned.csv

In [22]:
historic_olympic_data.to_csv('../csv/olympic_data_cleaned.csv', index=False)